In [1]:
import os 

In [1]:
dir='<directory to store generated job scripts>'
# os.mkdir(dir)
# %rm <directory to store generated job scripts>

In [2]:
subdirs=['fp32-O0-Adam','fp32-O1-Adam','fp32-O2-Adam','fp32-O1-AdamW','fp32-O1-LAMB']

for subdir in subdirs:
    subdirfull=os.path.join(dir,subdir)
    if not os.path.isdir(subdirfull):
        os.mkdir(subdirfull)
    profs={'fw':'Forward','bw':'Backward','opt':'Optimizer'}
    filenames=[subdir+'-'+x+'.sh' for x in ['fw','bw','opt']]
    for fn in filenames:
        with open(os.path.join(subdirfull,fn),'w') as f:
            f.write('''#!/bin/bash
#SBATCH -J {fn}\n'''.format(fn='-'.join(fn.split('.')[0].split('-')[1:4])))
            f.write('''#SBATCH -C gpu
#SBATCH --gres=gpu:1
##SBATCH --exclusive
#SBATCH -t 04:00:00

# conda activate py37pytorch 
conda activate /global/cfs/cdirs/m1759/charlene/condaenvs/py3.7pt1.5cuda10.2.89
\n''')

            f.write('''#parameters
run_tag="{}"\n'''.format(fn.split('.')[0].split('-')[3]))
            f.write('''data_dir_prefix="<path to data directory>"
output_dir=$SCRATCH/mlperf-deepcam/PT-{fn}-$SLURM_JOBID
\n'''.format(fn=fn.split('.')[0]))

            f.write('''#create files
mkdir -p ${output_dir}

# Prepare the run directory
script_dir=<path to mlperf-deepcam/src/deepCam/>
cp -r ${script_dir}/architecture ${output_dir}/
cp -r ${script_dir}/data ${output_dir}/
cp -r ${script_dir}/utils ${output_dir}/
cp ${script_dir}/profile_hdf5_ddp.py ${output_dir}/
cp ${script_dir}/train_hdf5_ddp.py ${output_dir}/ \n''')
            
            f.write('''cp $0 ${{output_dir}}/{fn}
cd ${{output_dir}}
\n'''.format(fn=fn))

            f.write('''metrics="sm__cycles_elapsed.avg.per_second \\
sm__cycles_elapsed.avg \\
sm__inst_executed_pipe_tensor.sum \\
sm__sass_thread_inst_executed_op_fadd_pred_on.sum \\
sm__sass_thread_inst_executed_op_ffma_pred_on.sum \\
sm__sass_thread_inst_executed_op_fmul_pred_on.sum \\
sm__sass_thread_inst_executed_op_hadd_pred_on.sum \\
sm__sass_thread_inst_executed_op_hfma_pred_on.sum \\
sm__sass_thread_inst_executed_op_hmul_pred_on.sum \\
dram__bytes.sum \\
lts__t_bytes.sum \\
l1tex__t_bytes.sum "

echo 'python version' `which python`

for metric in ${metrics}; do

    profilecmd="/usr/common/software/cuda/11.0.167/bin/nv-nsight-cu-cli --profile-from-start off --metrics ${metric} --csv --kernel-base demangled"

    srun -n 1 --cpu_bind=cores  -u \\
        ${profilecmd} \\
        `which python` -u ./profile_hdf5_ddp.py \\
        --wireup_method "nccl-slurm-pmi" \\
        --run_tag ${run_tag} \\
        --data_dir_prefix ${data_dir_prefix} \\
        --output_dir ${output_dir} \\
        --max_inter_threads 0 \\\n''')
            f.write('''        --optimizer "{optimizer}" \\\n'''.format(optimizer=fn.split('.')[0].split('-')[2]))
            f.write('''        --start_lr 1e-3 \\
        --num_warmup_steps 5 \\
        --num_profile_steps 1 \\\n''')
            f.write('''        --profile "{prof}" \\\n'''.format(prof=profs[fn.split('.')[0].split('-')[3]]))
            f.write('''        --weight_decay 1e-2 \\
        --amp_opt_level {optlevel} \\\n'''.format(optlevel=fn.split('.')[0].split('-')[1]))
            f.write('''        --local_batch_size 2 2>&1 > ${output_dir}/out.${metric} 

done \n''')

NameError: name 'os' is not defined